# Subgraph Isomorphism

### Useful References

* [Subgraph Isomorphism Problem](https://en.wikipedia.org/wiki/Subgraph_isomorphism_problem#CITEREFUllmann2010)
* [Original Algorithm](https://dl.acm.org/doi/pdf/10.1145/321921.321925)